In [1]:
import numpy as np
import time
import theano

print('blas.ldflags=', theano.config.blas.ldflags)

A = np.random.rand(1000, 10000).astype(theano.config.floatX)
B = np.random.rand(10000, 1000).astype(theano.config.floatX)
np_start = time.time()
AB = A.dot(B)
np_end = time.time()
X, Y = theano.tensor.matrices('XY')
mf = theano.function([X, Y], X.dot(Y))
t_start = time.time()
tAB = mf(A, B)
t_end = time.time()
print("NP time: %f[s], theano time: %f[s] (times should be close when run on CPU!)" % (
np_end - np_start, t_end - t_start))
print("Result difference: %f" % (np.abs(AB - tAB).max(), ))



('blas.ldflags=', '-L/home/nikasa/anaconda2/lib -lmkl_intel_lp64 -lmkl_intel_thread -lmkl_core -liomp5 -lpthread -lm -lm -Wl,-rpath,/home/nikasa/anaconda2/lib')
NP time: 0.171745[s], theano time: 0.207018[s] (times should be close when run on CPU!)
Result difference: 0.000000


In [ ]:
import sys
print (sys.version)

In [ ]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in xrange(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')


In [41]:
##Deconvolutional
import theano
import theano.tensor as T
import numpy as np

xfilter = np.asarray([[1,2,1],[1,0,1],[0,2,1]]).astype('float32')
xfilter = np.expand_dims(xfilter, axis=0)
xfilter = np.expand_dims(xfilter, axis=0)

filters = theano.shared(xfilter)

inp1 = T.tensor4(dtype='float32')

subsampled_convolution = T.nnet.conv2d(inp1, filters, border_mode='valid', subsample=(2, 2))

inp2 = T.tensor4(dtype='float32')
shp = inp2.shape
upsample = T.zeros((shp[0], shp[1], shp[2]*2, shp[3]*2), dtype=inp2.dtype)
upsample = T.set_subtensor(upsample[:, :, :2, :2], inp2)
upsampled_convolution = T.nnet.conv2d(upsample,
     filters.dimshuffle(1, 0, 2, 3)[:, :, ::-1, ::-1], border_mode='full')

f1 = theano.function([inp1], subsampled_convolution)
f2 = theano.function([inp2], upsampled_convolution)

x = np.asarray([[1,0,2,3,2],[2,1,1,0,1],[0,0,1,2,3],[2,3,1,0,2],[3,2,1,0,2]]).astype(np.float32)
x = np.expand_dims(x, axis=0)
x = np.expand_dims(x, axis=0)

f1x = f1(x)
y = f1x#np.random.randn(*f1x.shape).astype(np.float32)
f2y = f2(y)

#p1 = np.dot(f1x.ravel(), y.ravel())
#p2 = np.dot(x.ravel(), f2y[:, :, :, ::-1].ravel())

#print p1 - p2
print x
print xfilter
print f1x
print f2y
x = f2y
f1x = f1(x)
print f1x
#p1 = np.dot(f1x.ravel(), y.ravel())
#p2 = np.dot(x.ravel(), f2y.ravel())


[[[[ 1.  0.  2.  3.]
   [ 2.  1.  1.  0.]
   [ 0.  0.  1.  2.]
   [ 2.  3.  1.  0.]]]]
[[[[ 1.  2.  1.]
   [ 1.  0.  1.]
   [ 0.  2.  1.]]]]
[[[[ 5.]]]]
[[[[  5.  15.  10.   0.]
   [ 10.  20.  15.   5.]
   [ 10.  20.  20.  10.]
   [  5.  15.  15.   5.]]]]
[[[[ 130.]]]]


DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/k/AppData/Local/Theano/compiledir_Windows-7-6.1.7600-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64/tmp1mx_hy/f95c7354fe40c8eead82d6e815994d98.lib and object C:/Users/k/AppData/Local/Theano/compiledir_Windows-7-6.1.7600-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.11-64/tmp1mx_hy/f95c7354fe40c8eead82d6e815994d98.exp



In [3]:
xfilter = np.asarray([[1,2,1],[1,0,1],[0,2,1]]).astype('float32')
xfilter = np.expand_dims(xfilter, axis=0)
xfilter = np.expand_dims(xfilter, axis=0)

x = np.asarray([[1,0,2,3],[2,1,1,0],[0,0,1,2],[2,3,1,0]]).astype(np.float32)
x = np.expand_dims(x, axis=0)
x = np.expand_dims(x, axis=0)


In [12]:
x = np.asarray([1,2,0,1,0,1,1,2,1])
a1 = np.asarray([1,0,2,2,1,1,0,0,1])
a2 = np.asarray([0,2,3,1,1,0,0,1,2])
a3 = np.asarray([2,1,1,0,0,1,2,3,1])
a4 = np.asarray([1,1,0,0,1,2,3,1,0])
print np.dot(x, a1), np.dot(x,a2), np.dot(x, a3), np.dot(x, a4)


5 9 14 10
